In [1]:
import pandas as pd
labels_df = pd.read_csv('../UrbanSound8K/metadata/UrbanSound8K.csv')
print(labels_df.head)
labels_df = labels_df[labels_df['fold']==1][['slice_file_name', 'classID']]
labels_df['slice_file_name'] = '/openbayes/home/UrbanSound8K/audio/fold1/' + labels_df['slice_file_name']
print(labels_df.head)
df_set = labels_df.set_index('slice_file_name')
df_dict = df_set['classID'].to_dict()

<bound method NDFrame.head of          slice_file_name    fsID       start         end  salience  fold  \
0       100032-3-0-0.wav  100032    0.000000    0.317551         1     5   
1     100263-2-0-117.wav  100263   58.500000   62.500000         1     5   
2     100263-2-0-121.wav  100263   60.500000   64.500000         1     5   
3     100263-2-0-126.wav  100263   63.000000   67.000000         1     5   
4     100263-2-0-137.wav  100263   68.500000   72.500000         1     5   
...                  ...     ...         ...         ...       ...   ...   
8727     99812-1-2-0.wav   99812  159.522205  163.522205         2     7   
8728     99812-1-3-0.wav   99812  181.142431  183.284976         2     7   
8729     99812-1-4-0.wav   99812  242.691902  246.197885         2     7   
8730     99812-1-5-0.wav   99812  253.209850  255.741948         2     7   
8731     99812-1-6-0.wav   99812  332.289233  334.821332         2     7   

      classID             class  
0           3          

In [1]:
import pandas as pd
labels_df = pd.read_csv('../UrbanSound8K/metadata/UrbanSound8K.csv')
df_set = labels_df.set_index('slice_file_name')
df_dict = df_set['classID'].to_dict()

In [2]:
import numpy as np
import laion_clap
import glob
import json
import torch

In [5]:
class_index_dict = {'air_conditioner': 0, 'car_horn': 1 , 'children_playing': 2 , 'dog_bark': 3, 'drilling': 4, 'engine_idling': 5 , 'gun_shot': 6 , 'jackhammer': 7 , 'siren': 8, 'street_music': 9}

device = torch.device('cuda:0')

test_dir = '../UrbanSound8K/audio/fold1'

# Load the model
model = laion_clap.CLAP_Module(enable_fusion=False, device=device)
model.load_ckpt('../CLAP/630k-audioset-best.pt')

audio_files = []
with open('../output.txt', 'r') as file:
    for line in file:
        line = line.strip()
        audio_files.append(line)

temp = []
with open('../test.txt', 'r') as file:
    for line in file:
        line = line.strip()
        temp.append(line)
# audio_files = sorted(glob.glob(test_dir + '/**/*.wav', recursive=True))
# print(audio_files)
ground_truth_idx = [df_dict[af] for af in temp]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load the specified checkpoint ../CLAP/630k-audioset-best.pt from users.
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.weight

In [6]:
with torch.no_grad():
    ground_truth = torch.tensor(ground_truth_idx).view(-1, 1)

    # Get text features
    all_texts = ["This is a sound of " + t for t in class_index_dict.keys()]
    text_embed = model.get_text_embedding(all_texts)
    audio_embed = model.get_audio_embedding_from_filelist(x=audio_files)

    ranking = torch.argsort(torch.tensor(audio_embed) @ torch.tensor(text_embed).t(), descending=True)
    preds = torch.where(ranking == ground_truth)[1]
    preds = preds.cpu().numpy()

    metrics = {}
    metrics[f"mean_rank"] = preds.mean() + 1
    metrics[f"median_rank"] = np.floor(np.median(preds)) + 1
    for k in [1, 5, 10]:
        metrics[f"R@{k}"] = np.mean(preds < k)
    # map@10
    metrics[f"mAP@10"] = np.mean(np.where(preds < 10, 1 / (preds + 1), 0.0))

    print(
        f"Zeroshot Classification Results: "
        + "\t".join([f"{k}: {round(v, 4):.4f}" for k, v in metrics.items()])
    )

Zeroshot Classification Results: mean_rank: 1.5181	median_rank: 1.0000	R@1: 0.7626	R@5: 0.9684	R@10: 1.0000	mAP@10: 0.8534
